## Birdsound Fieldguide for Stockton

In [ ]:
# Query Xeno-Canto for recordings of a specific bird species
# Filter recordings by type (call or song)
# Download and save them in downloads/{species}/{type}/ directories

In [2]:
import requests
import os
from pydub import AudioSegment

WESTERN_LOCATIONS = [
    "California", "Oregon", "Washington", "Nevada", "Arizona",
    "Utah", "New Mexico", "Colorado", "Idaho", "Montana",
    "Wyoming", "Alaska"
]

def fetch_and_save_recordings(bird_name, base_dir="/path/to/output/folder", limit=30):
    query_url = f"https://xeno-canto.org/api/2/recordings?query={bird_name.replace(' ', '+')}"
    downloaded = 0
    page = 1

    while downloaded < limit:
        response = requests.get(f"{query_url}&page={page}")
        if response.status_code != 200:
            print(f"Failed to fetch data on page {page}")
            break

        data = response.json()
        recordings = data.get("recordings", [])
        if not recordings:
            break

        for rec in recordings:
            if downloaded >= limit:
                break

            file_type = rec.get("type", "").lower()
            if file_type not in ["call", "song"]:
                continue

            location = rec.get("loc", "")
            if not any(loc in location for loc in WESTERN_LOCATIONS):
                continue

            file_url = rec['file'] if rec['file'].startswith("http") else f"https:{rec['file']}"
            file_name = file_url.split("/")[-1].replace(".mp3", "")
            
            species_dir = os.path.join(base_dir, bird_name.replace(" ", "_"))
            type_dir = os.path.join(species_dir, file_type)
            os.makedirs(type_dir, exist_ok=True)

            wav_path = os.path.join(type_dir, f"{file_name}.wav")
            if os.path.exists(wav_path):
                continue  # Skip if already converted

            # Download MP3 to temp path
            temp_mp3_path = os.path.join(type_dir, f"{file_name}.mp3")
            print(f"Downloading and converting {file_type} from {location}: {file_name}.mp3")

            audio_data = requests.get(file_url)
            if audio_data.status_code == 200:
                with open(temp_mp3_path, "wb") as f:
                    f.write(audio_data.content)

                # Convert to WAV
                try:
                    sound = AudioSegment.from_mp3(temp_mp3_path)
                    sound.export(wav_path, format="wav")
                    os.remove(temp_mp3_path)  # Clean up .mp3
                    downloaded += 1
                except Exception as e:
                    print(f"Conversion failed for {file_name}: {e}")
                    os.remove(temp_mp3_path)
            else:
                print(f"Failed to download: {file_url}")

        page += 1

# Example usage
fetch_and_save_recordings("Turdus migratorius")  # American robin

Failed to fetch data on page 3
